In [7]:
import os
import numpy as np
from tqdm import tqdm
import re
import json

with open('annotated_predictions.json', 'r') as f:
    annotated_data = json.loads(f.read())
    
with open('summary_vicuna.json', 'r') as f:
    summaries = json.loads(f.read())

In [19]:
annotated_data[0]['annotations'][0]['result'][1]

{'value': {'choices': ['bot_1 enjoys sewing as a hobby to keep the voices at bay.']},
 'id': 'flGcEQOBYN',
 'from_name': 'facts_s2',
 'to_name': 'chat',
 'type': 'choices',
 'origin': 'prediction'}

In [21]:
annotated_data[0]['data'].keys()

dict_keys(['dialogue', 's1', 's2', 'summary', 'dialog_id', 'session', 'is_annotated'])

In [23]:
annotated_data = {(data['data']['dialog_id'], data['data']['session']): data for data in annotated_data}
summaries = {(data['data']['dialog_id'], data['data']['session']): data for data in summaries}

In [62]:
dialog_ids = map(lambda x: x[0], filter(lambda x: x[1] == 'session_1', annotated_data.keys()))

sessions = ['session_1', 'session_2', 'session_3']

dataset = dict()

for dialog_id in dialog_ids:
    dataset[dialog_id] = dict()
    for session in sessions:
        session_data = dict()
        session_data['dialog'] = [data['author'] + ': ' + data['text'] 
                                            for data in annotated_data[(dialog_id, session)]['data']['dialogue']]
        speakers = {'facts_s1': 'bot_0', 'facts_s2': 'bot_1'}
        session_data['facts'] = dict()
        for fact in annotated_data[(dialog_id, session)]['annotations'][0]['result']:
            if fact['from_name'] in speakers:
                session_data['facts'][speakers.pop(fact['from_name'])] = fact['value']['choices']
        session_data['context'] = summaries[(dialog_id, session)]['annotations'][0]['result'][0]['value']['text'][0]
        dataset[dialog_id][session] = session_data

In [63]:
with open('dialog_dataset.json', 'w') as f:
    json.dump(dataset, f)

In [67]:
dataset['train:ordered_1234']['session_1']['facts']

{'bot_0': ['Bot_0 is enthusiastic about the Archey Center.',
  'Bot_0 enjoys going to the Archey Center every hour and kind of.',
  'Bot_0 likes to ride bikes.',
  'Bot_0 is excited about a new job.',
  'Bot_0 will be cleaning gutters outside.',
  'Bot_0 enjoys riding bikes for fun.'],
 'bot_1': ['bot_1 enjoys sewing as a hobby to keep the voices at bay.']}